# Step 7: V04 — Depth stability (epoch-to-epoch consistency)

Goal: check whether individual transit depths are consistent with each other.

Why this matters:
- Large epoch-to-epoch depth variation can indicate systematics, contamination, or that the signal is driven by a small number of events.
- A single “dominating” epoch is a common false-alarm failure mode for weak signals.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V04')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "chi2_reduced": 1.34,
    "d_mean_w_ppm": 248.12,
    "d_med_ppm": 248.73,
    "depth_mad_ppm": 44.39,
    "depth_scatter_ppm": 60.9,
    "dmm": 0.998,
    "dmm_abs": 0.002,
    "dom_frac": 0.217,
    "dom_ratio": 1.318,
    "dominating_epoch_index": 3,
    "dominating_epoch_time_btjd": 3582.990287,
    "expected_scatter_ppm": 19.93,
    "mean_depth": 0.000253,
    "mean_depth_ppm": 253.0,
    "method": "per_epoch_local_baseline",
    "n_transits_measured": 8,
    "rms_scatter": 0.2409,
    "s_max": 6.156,
    "s_median": 4.681,
    "std_depth": 6.1e-05
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V04: Depth stability
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_depth_stability
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='07_v04_depth_stability')
    run_path = run_out_dir / 'V04_depth_stability.png'
    docs_path = (docs_out_dir / 'V04_depth_stability.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_depth_stability(r, ax=ax)
    ax.set_title('V04: Depth stability')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/07_v04_depth_stability/V04_depth_stability.png`

![V04: Depth stability](../artifacts/tutorial_toi-5807-incremental/07_v04_depth_stability/V04_depth_stability.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/07_v04_depth_stability/V04_depth_stability.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/07_v04_depth_stability/V04_depth_stability.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** depth scatter ∼60.9 ppm vs expected ∼19.93 ppm; dominating fraction ≈0.217; χ²ᵣᵉᵈ ≈1.34.
- **Why it’s useful:** checks the “one bad transit” failure mode; here no single epoch dominates the signal.
- **Interpretation:** some epoch-to-epoch variability is present (likely noise/systematics at this depth), but not catastrophic.
- **Next step:** V05 (transit shape: U vs V).

</details>
